In [1]:
import torch
from networks import FullyConnected, SPU
import numpy as np
from time import time
from itertools import product
from deeppoly_torch import DeepPolyVerifierTorch

DEVICE = "cpu"
INPUT_SIZE = 28

VERBOSE = False

class Args:
    net: str
    spec: str

    def __init__(self, net: str, spec: str):
        self.net = net
        self.spec = spec


def analyze(net: DeepPolyVerifierTorch, inputs: torch.FloatTensor, eps: float, true_label: int) -> str:
    """Returns "verified" if the returned label is the same for all points in a L-infinite epsilon-ball around an input point, and "not verified" otherwise.

    Args:
        net (torch.nn.Module): [description]
        inputs (torch.FloatTensor): [description]
        eps (float): [description]
        true_label (int): [description]

    Returns:
        [str]: Returns "verified" or "not verified"
    """
    start = time()

    """
    1. Transform initial input
    """
    initial_lower_bounds = torch.max(inputs - eps, torch.tensor(0.0)).to(DEVICE)
    initial_upper_bounds = torch.min(inputs + eps, torch.tensor(1.0)).to(DEVICE)

    """
    2. Propagate the domain through the network
    """

    with torch.no_grad():
        out, lower_bounds, upper_bounds = net(inputs, initial_lower_bounds, initial_upper_bounds)
    
    pred_label = out.max(dim=0)[1].item()
    assert pred_label == true_label



    """
    3. Verify
    """
    if VERBOSE:
        print(f"Upper bounds: {upper_bounds}")
        print(f"Lower bounds true label: {lower_bounds[true_label]}")
    
    verified = sum((lower_bounds[true_label] > upper_bounds).int()) == 9

    end = time()

    print(f"Propagation done in {round(end - start, 3)}!")

    if verified:
        return verified
    
    """
    4. Backsubstitution if not verified by simple forward propagation
    """
    order = None
    with torch.no_grad():
        lower_bounds, upper_bounds = net.backsubstitute(true_label=true_label, order=order)
    
    verified = (lower_bounds.detach().numpy() > 0).all()
    end = time()

    print(f"Backsubstitution done in {round(end - start, 3)}!")

    return verified


args = Args(
    net="net0_fc4",
    spec="../test_cases/net0_fc4/example_img0_0.03300.txt"
)

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])

if args.net.endswith('fc1'):
        net = FullyConnected(DEVICE, INPUT_SIZE, [50, 10]).to(DEVICE)
        deeppoly_verifier = DeepPolyVerifierTorch(
            DEVICE, INPUT_SIZE, [50, 10], verbose=VERBOSE).to(DEVICE)
elif args.net.endswith('fc2'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 50, 10]).to(DEVICE)
    deeppoly_verifier = DeepPolyVerifierTorch(
        DEVICE, INPUT_SIZE, [100, 50, 10], verbose=VERBOSE).to(DEVICE)
elif args.net.endswith('fc3'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 10]).to(DEVICE)
    deeppoly_verifier = DeepPolyVerifierTorch(
        DEVICE, INPUT_SIZE, [100, 100, 10], verbose=VERBOSE).to(DEVICE)
elif args.net.endswith('fc4'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 50, 10]).to(DEVICE)
    deeppoly_verifier = DeepPolyVerifierTorch(
        DEVICE, INPUT_SIZE, [100, 100, 50, 10], verbose=VERBOSE).to(DEVICE)
elif args.net.endswith('fc5'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [
                            100, 100, 100, 100, 10]).to(DEVICE)
    deeppoly_verifier = DeepPolyVerifierTorch(DEVICE, INPUT_SIZE, [
        100, 100, 100, 100, 10], verbose=VERBOSE).to(DEVICE)
else:
    assert False

net.load_state_dict(torch.load('../mnist_nets/%s.pt' %
                    args.net, map_location=torch.device(DEVICE)))

inputs = torch.FloatTensor(pixel_values).view(
    1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

if VERBOSE:
    print(net)

outs = net(inputs)
pred_label = outs.max(dim=1)[1].item()
assert pred_label == true_label

deeppoly_verifier.load_weights(net=net)

if analyze(deeppoly_verifier, inputs, eps, true_label):
    print('verified')
else:
    print('not verified')


FullyConnected(
  (layers): Sequential(
    (0): Normalization()
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=784, out_features=100, bias=True)
    (3): SPU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): SPU()
    (6): Linear(in_features=100, out_features=50, bias=True)
    (7): SPU()
    (8): Linear(in_features=50, out_features=10, bias=True)
  )
)

Current layer: Normalization()
	--> Skipping layer

Current layer: Flatten(start_dim=1, end_dim=-1)
	--> Skipping layer

Current layer: DeepPolyLinearTransformer(
  (layer): Linear(in_features=784, out_features=100, bias=True)
)
	Lower: tensor([-3.4511e+00, -3.6258e+00, -8.0352e+00, -5.5865e+00, -3.2474e+00,
        -1.4613e+00, -3.7203e+00, -3.0594e+00, -7.2055e+00,  1.2138e-01,
         1.9342e+00, -7.5876e+00, -5.5610e+00, -3.7362e+00, -3.1240e+00,
        -2.2335e+00, -1.1815e+01, -1.0023e+00, -9.6626e-01, -9.2849e-01,
         8.0011e-01, -2.9215e+00, -5.5636e+00, -1.0043e+01, -1.0899

# TESTS

In [1]:
layer_shape = (1,1,3,3)

In [5]:
[
    [
        [
            [0., 0., 0.],
            [0., 0., 0.],
            [0., 0., 0.]
        ]
    ]
]

np.zeros(layer_shape).flatten()


array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
tuple(list(map(lambda x: list(x), zip(
    *product(*map(range, layer_shape))))) * 2)


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])

In [13]:
indices = tuple(list(map(lambda x: list(x), zip(*product(*map(range, layer_shape))))))

In [14]:
a = np.arange(16).reshape((4,4))
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [17]:
a[::-1]

array([[12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])

In [3]:
a = [1,2,3,4]
a[:-1][::-1]

[3, 2, 1]

In [14]:
np.eye(4).reshape((1,1,4,4)) / 13.05

array([[[[0.07662835, 0.        , 0.        , 0.        ],
         [0.        , 0.07662835, 0.        , 0.        ],
         [0.        , 0.        , 0.07662835, 0.        ],
         [0.        , 0.        , 0.        , 0.07662835]]]])

In [8]:
a = np.ones((5, 2))
b = np.array([5, 5, 5, 5, 5])

print(f"a: {a}")
print(f"Shape of a: {a.shape}")
print(f"Shape of b: {b.shape}")
print(f"Shape of a + b: {(a + b.reshape(-1,1)).shape}")
print(f"a + b: {a + b.reshape(-1,1)}")

a: [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Shape of a: (5, 2)
Shape of b: (5,)
Shape of a + b: (5, 2)
a + b: [[6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]]


In [7]:
a = np.zeros((1,1,5, 5))
np.zeros((25,)).reshape((1,1,5,5))


array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]])

In [2]:
[1,2,3][:-1]

[1, 2]

In [7]:
np.zeros((4,4)).reshape((1,1,2,2,1,1,2,2))

array([[[[[[[[0., 0.],
             [0., 0.]]]],



          [[[[0., 0.],
             [0., 0.]]]]],




         [[[[[0., 0.],
             [0., 0.]]]],



          [[[[0., 0.],
             [0., 0.]]]]]]]])

In [5]:
shape = (1, 1, 28, 28) * 2
shape[4:]


(1, 1, 28, 28)

In [3]:
(shape[2] ** 2,)

(784,)

# `DeepPoly` test

In [ ]:
l1 = torch.nn.Linear(2, 2, device='cpu')
l1.weight = torch.Tensor([[1, 1], [1, -1]])
l1.bias = torch.Tensor([0, 0])

l2 = torch.nn.Linear(2, 2, device='cpu')
l2.weight = torch.Tensor([[1, 1], [1, -1]])
l2.bias = torch.Tensor([-0.5, 0])

l3 = torch.nn.Linear(2, 2, device='cpu')
l3.weight = torch.Tensor([[-1, 1], [0, 1]])
l3.bias = torch.Tensor([3, 0])

net = torch.nn.Sequential(l1, SPU, l2, SPU, l3, device='cpu')


input = torch.Tensor([0, 0])
eps = 1

verifier = DeepPolyVerifierTorch(net=net, inputs=input, eps=eps, true_label=0)
verifier.verify()



In [2]:
a = np.ones((5,1))
a

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [3]:
a[:, 3]

IndexError: index 3 is out of bounds for axis 1 with size 1

In [2]:
m = torch.nn.Linear(in_features=2, out_features=1)

In [6]:
m.weight.detach().numpy().shape
m.bias.detach().numpy().shape

(1,)